# RCT-FACTS on EIS
This notebook will check, install and run [RCT tools](https://radical-cybertools.github.io/) and [FACTS](https://fact-sealevel.readthedocs.io/en/latest/index.html) locally on this machine.

In [ ]:
import os
import sys
import time
import datetime
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt

Check if RCT tools are installed

In [ ]:
check_rct = !radical-stack
if 'radical.entk' in ' '.join(check_rct):
    print('RCT tools are installed')
else:
    rct_tools = ['saga', 'gtod', 'pilot', 'entk']
    for tool in rct_tools:
        print(f'Installing radical.{tool} is in progress')
        cmd_out = !pip install radical.{tool}
    print('Done')

!radical-stack

Run FACTS dummy experiments using RCT tools.

In [ ]:
# do not use animated output in notebooks
os.environ['RADICAL_LOG_LVL'] ='DEBUG'
os.environ['RADICAL_REPORT_ANIME'] = 'False'

In [ ]:
# Get the current working directory
current_dir = os.getcwd()
facts_dir = os.path.dirname(os.path.dirname(current_dir))

In [ ]:
from radical.entk import Pipeline, Stage, Task, AppManager

task_path = f'{facts_dir}/modules/fair/temperature'
task_env_cmd = os.path.expanduser('~') + '/ve/facts_eis_demo/bin/activate'

def generate_fair_temp_pipeline(pipeline_id):
    # Create a Pipeline object
    p = Pipeline()

    # Create A single stage for Preporcessing and Fitting tasks
    # as these tasks can run concurrently with no dependencies
    s0 = Stage()
    t0 = Task()
    t0.name = 'fair.preprocessing'
    t0.cpu_reqs = {'cpu_processes':1}
    t0.pre_exec = [f'. {task_env_cmd}']
    t0.executable = f'python {task_path}/fair_temperature_preprocess.py --pipeline_id {pipeline_id}'
    t0.upload_input_data = [f'{task_path}/rcmip/rcmip-emissions-annual-means-v5-1-0.csv']

    t1 = Task()
    t1.name = 'fair.fitting'
    t1.cpu_reqs = {'cpu_processes':1}
    t1.pre_exec = [f'. {task_env_cmd}']
    t1.executable = f'python {task_path}/fair_temperature_fit.py --pipeline_id {pipeline_id}'
    t1.upload_input_data = [f'{task_path}/parameters/fair_ar6_climate_params_v4.0.nc']
    s0.add_tasks([t0, t1])

    # Create A seprate Projecting Stage which runs after stage 0 outputs are produced
    s1 = Stage()
    t2 = Task()
    t2.name = 'fair.projectting'
    t2.cpu_reqs = {'cpu_processes':1}
    t2.pre_exec = [f'. {task_env_cmd}']
    t2.executable = f'python {task_path}/fair_temperature_project.py --pipeline_id {pipeline_id}'
    t2.upload_input_data = [f'{task_path}/{pipeline_id}_preprocess.pkl',
                            f'{task_path}/{pipeline_id}_fit.pkl']

    t2.download_output_data = ['0_climate.nc', '0_gsat.nc', '0_ohc.nc', '0_oceantemp.nc']
    s1.add_tasks(t2)

    # Create Projecting Stage
    s2 = Stage()
    t3 = Task()
    t3.name = 'fair.postprocessing'
    t3.cpu_reqs = {'cpu_processes':1}
    t3.pre_exec = [f'. {task_env_cmd}']
    t3.executable = f'python {task_path}/fair_temperature_postprocess.py --pipeline_id {pipeline_id}'
    s2.add_tasks(t3)

    p.add_stages([s0, s1, s2])

    return p

In [ ]:
pipelines = []
for i in range(1):
    pipe = generate_fair_temp_pipeline(pipeline_id=i)
    pipelines.append(pipe)

In [ ]:
# Create Application Manager
appman = AppManager()

# Create a dictionary describe four mandatory keys:
# resource, walltime, and cpus
# resource is 'local.localhost' to execute locally
res_dict = {'resource': 'local.localhost', 'walltime': 30, 'cpus': 8,}
# Assign resource request description to the Application Manager
appman.resource_desc = res_dict

In [ ]:
# Assign the workflow as a set or list of Pipelines to the Application Manager
# Note: The list order is not guaranteed to be preserved
appman.workflow = pipelines
# Run the Application Manager
appman.run()

# Plotting the FittedISMIP GrIS Module results

In [ ]:
year = 2100 # Default year was 2150
module = 'GrIS'
module_set = 'FittedISMIP'
sheet = 'GIS'

# load fair temperation gsat data and projection data
fair_temp = f'{current_dir}/0_gsat.nc'
module_data = f'{current_dir}/0_oceantemp.nc'

# Open .nc data sets and restrict to specific year
x_585 = (xr.open_dataset(fair_temp).squeeze(drop = True).surface_temperature.sel(years = year, drop = True)).values
y_585 = (xr.open_dataset(module_data).squeeze(drop = True).sea_level_change.sel(years = year, drop = True)/10).values
quantile_spp585 = np.quantile(y_585,[.05,.17,.5,.83,.95])

plt.title(f'{module_set}/{module} - {sheet}')
plt.xlabel('FAIR GSAT')
plt.ylabel(f'{module_set.capitalize()}/{module} [cm]')

# Plot Data
plt.scatter(x_585, y_585, s=3, label=f'ssp585: {quantile_spp585}',color='blue')
plt.legend(loc="upper left")

The cell above displays the output generated by running a sample pipeline experiment using `Radical.entk` for the FACTS workflow. This pipeline consists of a single stage containing a single task named `preprocess.task1`.

When the task dependencies are resolved by EnTK, it is then handed over to the `Radical.pilot` runtime system. In this phase, the task goes through the following states: `SCHEDULED`, `SUBMITTED`, and eventually, `EXECUTED`. These transitions occur on the designated computing resources and in this case we used the `localhost` (i.e. the current virtual machine itself).

Once all of the tasks are executed, EnTK initiates the termination process for the allocated computing resources. As part of this finalization step, EnTK collects and compiles the necessary performance profiles and logs from the executed tasks.

Finally, it gracefully closes the session, ensuring that all resources are released, and the experiment is completed in an orderly manner.